In [13]:
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy.sparse as sparse
from scipy.linalg import eigh
import scipy.linalg
import matplotlib.pyplot as plt
from qutip import *
import math

In [14]:
#plt.rcParams['figure.figsize']=8.5,5.1
plt.rcParams['font.size']=20
plt.rcParams['lines.linewidth']=1.5
plt.rcParams['text.usetex']=True

In [15]:
#Construction of necessary elements of theta and phi Hilbert spaces

# For phi

N=100
a_phi=np.diag(np.sqrt(np.arange(1, N+1)), 1)
ad_phi=np.diag(np.sqrt(np.arange(1, N+1)), -1)
N_phi=ad_phi@a_phi
Id_N=np.eye(N+1)

# For theta

M=40
Id_M=np.eye(2*M+1)
n_theta=np.flipud(np.fliplr(np.diag(np.arange(-M,M+1), 0)))

#Values of equilibrium

phi_ext_val=np.pi
ng_val=0.5

#Qubit parameters 1 

E_Cs=1.0
E_Cj=1.0
E_L=1.0
E_J=1.0
alpha=(1j*(E_Cj/E_L)**(1.0/4.0))

# Qubit parameters 2

E_Cs_2=(1.0/20)
E_Cj_2=1.0
E_L_2=(1.0/16)
E_J_2=(1.0/5.0)
alpha2=(1j*(E_Cj_2/E_L_2)**(1.0/4.0))

#FOR EXPECTATION VALUES

#PHI
print("PHI")
print((E_Cj/E_L)**(1.0/4.0)*(a_phi+ad_phi))
phi=np.kron(Id_M,((E_Cj/E_L)**(1.0/4.0))*(a_phi+ad_phi))
print("kron PHI")
print(phi)
phi2=np.kron(Id_M,((E_Cj_2/E_L_2)**(1.0/4.0))*(a_phi+ad_phi))

#COS_THETA
costheta=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
costheta[0][2*M]+=1
costheta[2*M][0]+=1
print(costheta)
print("COS")
cos_theta=np.kron(costheta,Id_N)
print("kron COS")
print(cos_theta)

#N_PHI
N_phi_extended=np.kron(Id_M,N_phi)

#N_theta
n_theta_extended=np.kron(n_theta,Id_N)


#FOR SAVING VALUES OF CALCULATIONS

#1ST VALUES

ng1=np.arange(0.48,0.53,0.04)
print(ng1)
a=np.pi-(np.pi*0.04)
b=np.pi+(np.pi*0.04 + 0.01)
phi_ext1=np.arange(a,b,(0.04*np.pi)*2)
print(phi_ext1)
Energies=[]
Eigenvectors=[]

Expvals_costheta_NG=[]
Expvals_phi_NG=[]
Expvals_N_phi_NG=[]
Expvals_n_theta_NG=[]

Expvals_costheta_PHIEXT=[]
Expvals_phi_PHIEXT=[]
Expvals_N_phi_PHIEXT=[]
Expvals_n_theta_PHIEXT=[]

#2ND VALUES

ng2=np.arange(0.48,0.53,0.04)
phi_ext2=np.arange(a,b,(0.04*np.pi)*2)
Energies2=[]
Eigenvectors2=[]

Expvals2_costheta_NG=[]
Expvals2_phi_NG=[]
Expvals2_N_phi_NG=[]
Expvals2_n_theta_NG=[]

Expvals2_costheta_PHIEXT=[]
Expvals2_phi_PHIEXT=[]
Expvals2_N_phi_PHIEXT=[]
Expvals2_n_theta_PHIEXT=[]


PHI
[[ 0.          1.          0.         ...  0.          0.
   0.        ]
 [ 1.          0.          1.41421356 ...  0.          0.
   0.        ]
 [ 0.          1.41421356  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          9.94987437
   0.        ]
 [ 0.          0.          0.         ...  9.94987437  0.
  10.        ]
 [ 0.          0.          0.         ...  0.         10.
   0.        ]]
kron PHI
[[ 0.          1.          0.         ...  0.          0.
   0.        ]
 [ 1.          0.          1.41421356 ...  0.          0.
   0.        ]
 [ 0.          1.41421356  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          9.94987437
   0.        ]
 [ 0.          0.          0.         ...  9.94987437  0.
  10.        ]
 [ 0.          0.          0.         ...  0.         10.
   0.        ]]
[[0. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0

In [16]:
# Construction of the 0-pi Hamiltonian

def Hamiltonian_0pi(EJ,ECS,ECJ,EL,n,phi,alph):
    #Initial
    H=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Free Hamiltonian construction
    H1=4*ECS*((n_theta-n*Id_M)@(n_theta-n*Id_M))
    H_theta=np.kron(H1,Id_N)
    H2=2*np.sqrt(ECJ*EL)*(N_phi+0.5*Id_N)
    H_phi=np.kron(Id_M,H2)
    H_0=H_theta+H_phi
    #H_0=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Interaction Hamiltonian construction
    cos_theta_ext=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
    #cos_theta=np.diag(np.ones(2*M-1),2)+np.diag(np.ones(2*M-1),-2)
    cos_theta_ext[0][2*M]+=1
    cos_theta_ext[2*M][0]+=1
    #cos_phi=(0.5*((displace(N+1,alph)*(math.e**(-1j*phi/2)))+(displace(N+1,-alph)*(math.e**(1j*phi/2)))))*np.diag(np.ones(N+1),0)
    cos_phi=0.5*(scipy.linalg.expm(alph*(a_phi+ad_phi)-(Id_N*1j*phi/2))+scipy.linalg.expm(-alph*(a_phi+ad_phi)+(Id_N*1j*phi/2)))
    H_I=EJ*np.kron(cos_theta_ext,cos_phi.real)#SIN EL2 
    #H_I=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Total Hamiltonian
    H+=H_0-H_I
    return H

In [17]:
def calculate_expval(vect,operator):
    return np.conj(vect)@operator@vect

In [18]:
Vprueba=[1,1,0]
print(Vprueba)
matrix=[[0,1,0],[1,0,1],[0,1,0]]
print(matrix)
a=calculate_expval(Vprueba,matrix)
print(a)


[1, 1, 0]
[[0, 1, 0], [1, 0, 1], [0, 1, 0]]
2


## $\text{1. Plot for } E_{Cs} = E_{Cj} = E_L = 1$, $\varphi_{ext}= \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = i$.

In [19]:
# Calculations 1

for n1 in ng1:
    print("Step for ng = ",n1)
    H1=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,n1,phi_ext_val,alpha)
    print("Hamiltonian built, len(H)=",len(H1)**2)
    #vals,vects=np.linalg.eigh(H)
    valsNG, vectsNG = eigsh(H1, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsNG[0]," and ",valsNG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsNG[:,0]," and ",vectsNG[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsNG[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsNG[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsNG[:,0],phi)
    Expval_phi_1=calculate_expval(vectsNG[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsNG[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsNG[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsNG[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsNG[:,1],n_theta_extended)
    #TOTALS
    Expvals_costheta_NG.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_NG.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_NG.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_NG.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for ng =  0.48
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  1.7049371654883658  and  1.8404210481619057
The eigenvectors of the GS and the 1st excited state are  [ 5.96086035e-18 -5.50179658e-18  2.44742670e-18 ...  3.18240178e-17
  2.69211054e-17 -1.55946855e-16]  and  [ 3.66048686e-19  2.48733260e-18  2.35816802e-19 ... -8.69005993e-17
 -8.53126069e-17 -2.68621083e-16]
Eigenvalues and eigenvectors calculated
Step for ng =  0.52
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  1.7049371654885863  and  1.8404210481619965
The eigenvectors of the GS and the 1st excited state are  [ 5.98165583e-20 -1.22596597e-19  5.72986535e-19 ...  2.80906956e-17
 -1.78216846e-17 -1.65818069e-16]  and  [-3.59356204e-18  1.19966610e-18  7.69039822e-18 ...  1.06640375e-16
 -1.73042372e-16 -3.30850858e-16]
Eigenvalues and eigenvectors calculated


In [20]:
# Calculations 1

for phi1 in phi_ext1:
    print("Step for phi_ext = ",phi1)
    H2=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,ng_val,phi1,alpha)
    print("Hamiltonian built, len(H)=",len(H2)**2)
    #vals,vects=np.linalg.eigh(H)
    valsPHI, vectsPHI = eigsh(H2, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsPHI[0]," and ",valsPHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsPHI[:,0]," and ",vectsPHI[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsPHI[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsPHI[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsPHI[:,0],phi)
    Expval_phi_1=calculate_expval(vectsPHI[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsPHI[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsPHI[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsPHI[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsPHI[:,1],n_theta_extended)
    #TOTALS
    Expvals_costheta_PHIEXT.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_PHIEXT.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_PHIEXT.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_PHIEXT.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for phi_ext =  3.015928947446201
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  1.7372294513264477  and  1.807426536608833
The eigenvectors of the GS and the 1st excited state are  [ 7.77300791e-18 -4.48690760e-18  3.73051002e-18 ...  7.39837418e-17
 -3.50893009e-17  3.06658070e-16]  and  [ 2.57742209e-18  9.44118232e-19  1.08996601e-18 ... -5.04184723e-18
  1.02197081e-17 -3.78835112e-19]
Eigenvalues and eigenvectors calculated
Step for phi_ext =  3.2672563597333846
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  1.73722945132658  and  1.8074265366083906
The eigenvectors of the GS and the 1st excited state are  [-2.95007169e-18  8.82544482e-18  7.55959726e-18 ... -3.72203396e-17
 -8.90598324e-17 -1.89662725e-16]  and  [-2.82870550e-19 -2.21207140e-18 -5.01957884e-18 ... -3.24918467e-17
 -9.42037802e-17 -1.00105618e-16]
Eigenvalues and eigenvectors calculated


EXPECTATION VALUES FOR THE FIRST SET OF VALUES FOR THE ENERGIES

In [21]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals_costheta_NG)
print("Exp values of phi varying ng are ",Expvals_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [8.370230585253709e-13, -8.465102433837611e-13, -9.996616486742187e-13, 1.0513638204182166e-12]
Exp values of phi varying ng are  [4.585683605166156e-13, -6.028557157741949e-13, -4.213691238742247e-13, 6.644066075726346e-13]
Exp values of N_phi varying ng are  [0.07986229011957371, 0.10106746341756831, 0.07986229011952216, 0.10106746341757725]
Exp values of n_theta varying ng are  [0.06655789331297032, 0.9120589135131805, 0.9334421066870839, 0.08794108648680518]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0787415241388527, -1.0653339214587174, -1.0787415241388096, 1.065333921458694]
Exp values of phi varying phi_ext are  [0.545523065500263, -0.5707353367711993, -0.5455230655000134, 0.5707353367711677]
Exp values of N_phi varying phi_ext are  [0.08632475911287885, 0.09241049853126437, 0.08632475911280996, 0.09241049853126258]
Exp values of n_theta varying phi_ext are  [0.49999999999693656, 0.5000000000030684, 0.50000

## $\text{2. Plot for } E_{Cs} = \frac{1}{20}$, $E_{CJ} = 1$, $E_L = \frac{1}{16}$, $\varphi_{ext} = \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = 2i$.

In [22]:
# Calculations 2

for n2 in ng2:
    print("Step for ng = ",n2)
    H3=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,n2,phi_ext_val,alpha2)
    print("Hamiltonian built, len(H)=",len(H3)**2)
    #vals2,vects2=np.linalg.eigh(H2)
    vals2NG, vects2NG = eigsh(H3, k=2, which='SA')
    #vals2,_=np.linalg.eigh(H3)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2NG[0]," and ",vals2NG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2NG[:,0]," and ",vects2NG[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2NG[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2NG[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2NG[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2NG[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2NG[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2NG[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2NG[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2NG[:,1],n_theta_extended)
    #TOTALS
    Expvals2_costheta_NG.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_NG.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_NG.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_NG.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for ng =  0.48
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  0.2713101816917999  and  0.27911378888679045
The eigenvectors of the GS and the 1st excited state are  [-3.60440371e-18 -1.19533975e-18  4.17141259e-18 ... -5.59094747e-17
 -6.41325800e-17  1.65074743e-16]  and  [-4.89976770e-18 -9.20651496e-18  6.38100580e-18 ...  3.17904563e-17
  3.28758393e-17 -2.35469244e-16]
Eigenvalues and eigenvectors calculated
Step for ng =  0.52
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  0.27131018169182053  and  0.2791137888868373
The eigenvectors of the GS and the 1st excited state are  [ 1.50783978e-17  8.86537952e-18 -1.55813482e-17 ...  3.63706140e-18
 -9.45414049e-17  7.03911484e-17]  and  [ 5.43815217e-18 -8.41322333e-20 -7.90571658e-18 ...  3.26208322e-18
  6.62357665e-17  2.09033496e-16]
Eigenvalues and eigenvectors calculated


In [23]:
# Calculations 2

for phi2 in phi_ext2:
    print("Step for phi_ext = ",phi2)
    H4=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi2,alpha2)
    print("Hamiltonian built, len(H)=",len(H4)**2)
    #vals,vects=np.linalg.eigh(H)
    vals2PHI, vects2PHI = eigsh(H4, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2PHI[0]," and ",vals2PHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2PHI[:,0]," and ",vects2PHI[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2PHI[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2PHI[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2PHI[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2PHI[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2PHI[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2PHI[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2PHI[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2PHI[:,1],n_theta_extended)
    #TOTALS
    Expvals2_costheta_PHIEXT.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_PHIEXT.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_PHIEXT.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_PHIEXT.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for phi_ext =  3.015928947446201
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  0.2734323639714423  and  0.276857231770308
The eigenvectors of the GS and the 1st excited state are  [-4.47042235e-17 -2.30751447e-17 -2.28528190e-17 ... -3.98442002e-17
  3.79898604e-17 -8.84340385e-17]  and  [ 5.59495115e-18  4.04806980e-18  8.67381150e-18 ... -3.44434151e-18
 -8.97475644e-17 -4.14051431e-17]
Eigenvalues and eigenvectors calculated
Step for phi_ext =  3.2672563597333846
Hamiltonian built, len(H)= 66928761
The eigenvalues of the GS and the 1st excited state are  0.27343236397145804  and  0.27685723177033705
The eigenvectors of the GS and the 1st excited state are  [-9.04997440e-18 -8.53295756e-19  6.62235984e-19 ...  3.99970612e-18
  5.95581147e-18  2.64839147e-16]  and  [ 1.51406261e-17  4.47695940e-18 -4.05303962e-18 ...  8.63118245e-18
  1.75443410e-17  3.33008902e-16]
Eigenvalues and eigenvectors calculated


EXPECTATION VALUES FOR THE SECOND SET OF VALUES FOR THE ENERGIES

In [24]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals2_costheta_NG)
print("Exp values of phi varying ng are ",Expvals2_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals2_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals2_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals2_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals2_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals2_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals2_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [-6.930788941058249e-12, 7.233367264271159e-12, -5.777564441127208e-12, 5.645678509197062e-12]
Exp values of phi varying ng are  [-1.3750060477274747e-12, 1.4549548436244549e-12, -1.0738418319613877e-12, 1.1595963509010828e-12]
Exp values of N_phi varying ng are  [0.043701345099840215, 0.04460021040568321, 0.04370134509982429, 0.04460021040567748]
Exp values of n_theta varying ng are  [0.01150802342722652, 0.9869239041721941, 0.9884919765727948, 0.013076095827774097]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0891265685512788, -1.074504178428609, -1.0891265685511835, 1.0745041784286546]
Exp values of phi varying phi_ext are  [0.21530721534569686, -0.22023059456228516, -0.2153072153457004, 0.22023059456230337]
Exp values of N_phi varying phi_ext are  [0.04396098632823184, 0.04425921542514188, 0.04396098632822998, 0.0442592154251441]
Exp values of n_theta varying phi_ext are  [0.5000000000006097, 0.4999999999994052, 